In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-06 14:52:33.288250: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=None)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_1_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

Epoch 24/1000
10/10 [==============================] - 0s 30ms/step - loss: 10.8596 - mean_squared_error: 10.8596 - rmse: 3.2954 - mean_absolute_error: 2.6217 - val_loss: 11.2827 - val_mean_squared_error: 11.2827 - val_rmse: 3.3590 - val_mean_absolute_error: 2.5052 - lr: 0.0010
Epoch 25/1000
10/10 [==============================] - 0s 29ms/step - loss: 10.6199 - mean_squared_error: 10.6199 - rmse: 3.2588 - mean_absolute_error: 2.5613 - val_loss: 10.7631 - val_mean_squared_error: 10.7631 - val_rmse: 3.2807 - val_mean_absolute_error: 2.4613 - lr: 0.0010
Epoch 26/1000
10/10 [==============================] - 0s 29ms/step - loss: 10.3018 - mean_squared_error: 10.3018 - rmse: 3.2096 - mean_absolute_error: 2.5667 - val_loss: 12.1277 - val_mean_squared_error: 12.1277 - val_rmse: 3.4825 - val_mean_absolute_error: 2.5753 - lr: 0.0010
Epoch 27/1000
10/10 [==============================] - 0s 30ms/step - loss: 9.9952 - mean_squared_error: 9.9952 - rmse: 3.1615 - mean_absolute_error: 2.4947 - val_

Epoch 54/1000
10/10 [==============================] - 0s 37ms/step - loss: 8.3874 - mean_squared_error: 8.3874 - rmse: 2.8961 - mean_absolute_error: 2.2510 - val_loss: 8.4600 - val_mean_squared_error: 8.4600 - val_rmse: 2.9086 - val_mean_absolute_error: 2.1930 - lr: 0.0010
Epoch 55/1000
10/10 [==============================] - 0s 31ms/step - loss: 8.9496 - mean_squared_error: 8.9496 - rmse: 2.9916 - mean_absolute_error: 2.3392 - val_loss: 9.1785 - val_mean_squared_error: 9.1785 - val_rmse: 3.0296 - val_mean_absolute_error: 2.2632 - lr: 0.0010
Epoch 56/1000
10/10 [==============================] - 0s 30ms/step - loss: 8.0570 - mean_squared_error: 8.0570 - rmse: 2.8385 - mean_absolute_error: 2.2471 - val_loss: 8.9452 - val_mean_squared_error: 8.9452 - val_rmse: 2.9909 - val_mean_absolute_error: 2.2275 - lr: 0.0010
Epoch 57/1000
10/10 [==============================] - 0s 30ms/step - loss: 8.8994 - mean_squared_error: 8.8994 - rmse: 2.9832 - mean_absolute_error: 2.3228 - val_loss: 7.3074

Epoch 84/1000
10/10 [==============================] - 0s 34ms/step - loss: 8.1354 - mean_squared_error: 8.1354 - rmse: 2.8523 - mean_absolute_error: 2.2318 - val_loss: 8.9811 - val_mean_squared_error: 8.9811 - val_rmse: 2.9968 - val_mean_absolute_error: 2.2125 - lr: 0.0010
Epoch 85/1000
10/10 [==============================] - 0s 35ms/step - loss: 8.5029 - mean_squared_error: 8.5029 - rmse: 2.9160 - mean_absolute_error: 2.2799 - val_loss: 7.5436 - val_mean_squared_error: 7.5436 - val_rmse: 2.7466 - val_mean_absolute_error: 2.1158 - lr: 0.0010
Epoch 86/1000
10/10 [==============================] - 0s 37ms/step - loss: 10.0399 - mean_squared_error: 10.0399 - rmse: 3.1686 - mean_absolute_error: 2.4852 - val_loss: 7.5312 - val_mean_squared_error: 7.5312 - val_rmse: 2.7443 - val_mean_absolute_error: 2.1213 - lr: 0.0010
Epoch 87/1000
10/10 [==============================] - 0s 35ms/step - loss: 9.6147 - mean_squared_error: 9.6147 - rmse: 3.1008 - mean_absolute_error: 2.4043 - val_loss: 11.9

Epoch 112/1000
10/10 [==============================] - 0s 24ms/step - loss: 6.2279 - mean_squared_error: 6.2279 - rmse: 2.4956 - mean_absolute_error: 1.9594 - val_loss: 8.9884 - val_mean_squared_error: 8.9884 - val_rmse: 2.9981 - val_mean_absolute_error: 2.2191 - lr: 2.5000e-04
Epoch 113/1000
10/10 [==============================] - 0s 24ms/step - loss: 6.2990 - mean_squared_error: 6.2990 - rmse: 2.5098 - mean_absolute_error: 1.9766 - val_loss: 9.2594 - val_mean_squared_error: 9.2594 - val_rmse: 3.0429 - val_mean_absolute_error: 2.2553 - lr: 2.5000e-04
Epoch 114/1000
10/10 [==============================] - 0s 24ms/step - loss: 5.9546 - mean_squared_error: 5.9546 - rmse: 2.4402 - mean_absolute_error: 1.8979 - val_loss: 9.1008 - val_mean_squared_error: 9.1008 - val_rmse: 3.0168 - val_mean_absolute_error: 2.2260 - lr: 2.5000e-04
Epoch 115/1000
10/10 [==============================] - 0s 23ms/step - loss: 6.4448 - mean_squared_error: 6.4448 - rmse: 2.5387 - mean_absolute_error: 2.0169 - 

...

Epoch 51/1000
10/10 [==============================] - 0s 30ms/step - loss: 8.4058 - mean_squared_error: 8.4058 - rmse: 2.8993 - mean_absolute_error: 2.2873 - val_loss: 9.0741 - val_mean_squared_error: 9.0741 - val_rmse: 3.0123 - val_mean_absolute_error: 2.2434 - lr: 0.0010
Epoch 52/1000
10/10 [==============================] - 0s 31ms/step - loss: 8.4472 - mean_squared_error: 8.4472 - rmse: 2.9064 - mean_absolute_error: 2.3062 - val_loss: 7.3662 - val_mean_squared_error: 7.3662 - val_rmse: 2.7141 - val_mean_absolute_error: 2.2099 - lr: 0.0010
Epoch 53/1000
10/10 [==============================] - 0s 30ms/step - loss: 10.1337 - mean_squared_error: 10.1337 - rmse: 3.1834 - mean_absolute_error: 2.5237 - val_loss: 7.7947 - val_mean_squared_error: 7.7947 - val_rmse: 2.7919 - val_mean_absolute_error: 2.0970 - lr: 0.0010
Epoch 54/1000
10/10 [==============================] - 0s 29ms/step - loss: 9.4579 - mean_squared_error: 9.4579 - rmse: 3.0754 - mean_absolute_error: 2.4359 - val_loss: 7.85

Epoch 81/1000
10/10 [==============================] - 0s 27ms/step - loss: 7.9316 - mean_squared_error: 7.9316 - rmse: 2.8163 - mean_absolute_error: 2.2398 - val_loss: 12.7851 - val_mean_squared_error: 12.7851 - val_rmse: 3.5756 - val_mean_absolute_error: 2.7610 - lr: 0.0010
Epoch 82/1000
10/10 [==============================] - 0s 26ms/step - loss: 7.9682 - mean_squared_error: 7.9682 - rmse: 2.8228 - mean_absolute_error: 2.2084 - val_loss: 9.9788 - val_mean_squared_error: 9.9788 - val_rmse: 3.1589 - val_mean_absolute_error: 2.3499 - lr: 0.0010
Epoch 83/1000
10/10 [==============================] - 0s 28ms/step - loss: 7.5018 - mean_squared_error: 7.5018 - rmse: 2.7390 - mean_absolute_error: 2.1557 - val_loss: 6.3866 - val_mean_squared_error: 6.3866 - val_rmse: 2.5272 - val_mean_absolute_error: 1.9323 - lr: 0.0010
Epoch 84/1000
10/10 [==============================] - 0s 28ms/step - loss: 9.1662 - mean_squared_error: 9.1662 - rmse: 3.0276 - mean_absolute_error: 2.3920 - val_loss: 6.56

Epoch 109/1000
10/10 [==============================] - 0s 29ms/step - loss: 6.6980 - mean_squared_error: 6.6980 - rmse: 2.5881 - mean_absolute_error: 2.0299 - val_loss: 10.7216 - val_mean_squared_error: 10.7216 - val_rmse: 3.2744 - val_mean_absolute_error: 2.5093 - lr: 2.5000e-04
Epoch 110/1000
10/10 [==============================] - 0s 29ms/step - loss: 6.3946 - mean_squared_error: 6.3946 - rmse: 2.5288 - mean_absolute_error: 1.9386 - val_loss: 10.5389 - val_mean_squared_error: 10.5389 - val_rmse: 3.2464 - val_mean_absolute_error: 2.4548 - lr: 2.5000e-04
Epoch 111/1000
10/10 [==============================] - 0s 31ms/step - loss: 6.3605 - mean_squared_error: 6.3605 - rmse: 2.5220 - mean_absolute_error: 1.9605 - val_loss: 10.2495 - val_mean_squared_error: 10.2495 - val_rmse: 3.2015 - val_mean_absolute_error: 2.4244 - lr: 2.5000e-04
Epoch 112/1000
10/10 [==============================] - 0s 31ms/step - loss: 6.4923 - mean_squared_error: 6.4923 - rmse: 2.5480 - mean_absolute_error: 1.9

10/10 [==============================] - 0s 29ms/step - loss: 6.0934 - mean_squared_error: 6.0934 - rmse: 2.4685 - mean_absolute_error: 1.9236 - val_loss: 10.4857 - val_mean_squared_error: 10.4857 - val_rmse: 3.2382 - val_mean_absolute_error: 2.4777 - lr: 1.2500e-04
Epoch 138/1000
10/10 [==============================] - 0s 28ms/step - loss: 6.2778 - mean_squared_error: 6.2778 - rmse: 2.5056 - mean_absolute_error: 1.9151 - val_loss: 9.6797 - val_mean_squared_error: 9.6797 - val_rmse: 3.1112 - val_mean_absolute_error: 2.3610 - lr: 1.2500e-04
Epoch 139/1000
10/10 [==============================] - 0s 27ms/step - loss: 6.4550 - mean_squared_error: 6.4550 - rmse: 2.5407 - mean_absolute_error: 1.9489 - val_loss: 10.3326 - val_mean_squared_error: 10.3326 - val_rmse: 3.2144 - val_mean_absolute_error: 2.4458 - lr: 1.2500e-04
Epoch 140/1000
10/10 [==============================] - 0s 28ms/step - loss: 5.9912 - mean_squared_error: 5.9912 - rmse: 2.4477 - mean_absolute_error: 1.8927 - val_loss: 1

10/10 [==============================] - 0s 25ms/step - loss: 6.4378 - mean_squared_error: 6.4378 - rmse: 2.5373 - mean_absolute_error: 1.9660 - val_loss: 10.6255 - val_mean_squared_error: 10.6255 - val_rmse: 3.2597 - val_mean_absolute_error: 2.4895 - lr: 6.2500e-05
Epoch 166/1000
 7/10 [====================>.........] - ETA: 0s - loss: 6.0466 - mean_squared_error: 6.0466 - rmse: 2.4590 - mean_absolute_error: 1.9150
Epoch 00166: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
10/10 [==============================] - 0s 23ms/step - loss: 5.9376 - mean_squared_error: 5.9376 - rmse: 2.4367 - mean_absolute_error: 1.8928 - val_loss: 10.3749 - val_mean_squared_error: 10.3749 - val_rmse: 3.2210 - val_mean_absolute_error: 2.4620 - lr: 6.2500e-05


In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)